In [61]:
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen as uReq
from collections import defaultdict
import csv


In [139]:
teams =  ['acadia', 'capebreton', 'dalhousie', 'memorial', "saintmarys", 'stfx', 'unb', 'upei',
    'bishops', 'concordia', 'laval', 'mcgill', 'uqam', 'algoma', 'brock', 'carleton', 
    'guelph', 'lakehead', 'laurentian', 'laurier', 'mcmaster', 'nipissing', 'ontariotech', 
    'ottawa', 'queens', 'ryerson', 'toronto', 'torontometropolitan', 'waterloo', 'western', 'windsor', 
    'york', 'alberta', 'brandon', 'calgary', 'lethbridge', 'macewan', 'manitoba', 'mountroyal', 
    'regina', 'saskatchewan', 'thompsonrivers', 'trinitywestern', 'ubc', 'ubcokanagan',
    'ufv', 'unbc', 'victoria', 'winnipeg']
teams=['acadia']
years = ['2022-23']

base_url = 'https://universitysport.prestosports.com/sports/wbkb/'
end_url = '?view=splits'
href_list = []
headers = ['SEASON', 'TEAM', 'SPLIT', 'GP', 'FGM', 'FGA', 'FG%', '3FGM', '3FGA','3FG%', 'FTM', 'FTA', 'FT%', 
           'DREB/G', 'OREB/G', 'REB/G', 'A/G','TO/G', 'STL/G', 'BLK/G','PF/G', 'PPG', 'OFF_EFF', 'NET_EFF']
rows = [headers]

In [140]:
for year in years:
    season = '20' + year[5:7]
    for team in teams:
        current_url = base_url + year + '/teams/' + team + end_url
        print(current_url)
        r = requests.get(current_url) #extract the HTML code from current url
        raw_html = r.content #get just the content from HTML code
        soup = BeautifulSoup(raw_html, 'html.parser') #create parser called soup that uses contents of raw_html
        tables = soup.findAll('table') #tables contains all table elements from current page
        
        max_len = 0
        index = 0
        
        #some split stat tables are in different locations, find it by finding table with no url links
        for i in range(len(tables)):
            tags = tables[i].findAll('a')
            print(len(tags))
            if len(tags) == 0:
                table_idx = i
            # if len(tags) > 0:
            #     url = tags[0].get('href', None)
            #     if "/boxscores/20" in url and len(tables[i]) > max_len:
            #         index = i
            #         max_len = len(tables[i])
        
        
        # if len(tables) > 0: #check that this team has data or current year
        #     index = 10 #specifies which table/stats to collect
        #     splitstats_tbl = tables[index]
        #     all_stats = splitstats_tbl.findAll('tr') #get all url's in table 

https://universitysport.prestosports.com/sports/wbkb/2022-23/teams/acadia?view=splits
1
5
26
15
15
15
15
15
15
24
0
24


In [141]:
table_idx

10

In [137]:
tables[12]

IndexError: list index out of range

In [101]:
def parse_values(string):
    temp_list = string.split(' ') #split into list, each ' ' is separate entry
    for idx in range(len(temp_list)): #loop through list, select first and last name, remove \n chars
        if len(temp_list[idx]) > 0 and temp_list[idx] != '\n':
            value = temp_list[idx] #remove \n from end of text
    try:
        if value == '-':
            value = 0
    except:
        value = np.nan
        
        
    return value


def parse_shooting(fraction_str):
    makes_atts = fraction_str.split('-')
    makes = float(makes_atts[0])
    attempts = float(makes_atts[1])
    return makes, attempts

idx = 1
for row in all_stats[1:len(all_stats)]:
    if idx not in [3,6,7,10]: #skip empty rows
        #print(row)
        stats = row.find_all("td") #extract text from every column item/table data

        #extract data from each column
        category = stats[0].text #split stat category
        gp = float(parse_values(stats[1].text))
        fgm, fga = parse_shooting(parse_values(stats[2].text))    
        fg_pct = float(parse_values(stats[3].text))
        fgm3, fga3 = parse_shooting(parse_values(stats[4].text))
        fg3_pct = float(parse_values(stats[5].text))
        ftm, fta = parse_shooting(parse_values(stats[6].text))
        ft_pct = float(parse_values(stats[7].text))
        reb_def = float(parse_values(stats[8].text))
        reb_off = float(parse_values(stats[9].text))
        reb = float(parse_values(stats[10].text))
        apg = float(parse_values(stats[11].text))
        to = float(parse_values(stats[12].text))
        stl = float(parse_values(stats[13].text))
        blk = float(parse_values(stats[14].text))
        pf = float(parse_values(stats[15].text))
        ppg = float(parse_values(stats[16].text))
        off_eff = float(parse_values(stats[17].text))
        net_eff = float(parse_values(stats[18].text))
        
        rows.append([season, team, category, gp, fgm, fga, fg_pct, fgm3, fga3, fg3_pct, ftm, fta, ft_pct,
                    reb_def, reb_off, reb, apg, to, stl, blk, pf, ppg, off_eff, net_eff])
        
    idx += 1

In [102]:
rows

[['SEASON',
  'TEAM',
  'SPLIT',
  'GP',
  'FGM',
  'FGA',
  'FG%',
  '3FGM',
  '3FGA',
  '3FG%',
  'FTM',
  'FTA',
  'FT%',
  'DREB/G',
  'OREB/G',
  'REB/G',
  'A/G',
  'TO/G',
  'STL/G',
  'BLK/G',
  'PF/G',
  'PPG',
  'OFF_EFF',
  'NET_EFF'],
 ['2023',
  'acadia',
  'Total',
  19.0,
  568.0,
  1331.0,
  42.7,
  159.0,
  436.0,
  36.5,
  256.0,
  318.0,
  80.5,
  10.5,
  29.4,
  39.9,
  16.6,
  15.3,
  9.3,
  3.2,
  17.8,
  81.6,
  0.981,
  0.175],
 ['2023',
  'acadia',
  'Exhibition',
  5.0,
  126.0,
  297.0,
  42.4,
  36.0,
  104.0,
  34.6,
  71.0,
  81.0,
  87.7,
  7.4,
  26.8,
  34.2,
  15.4,
  11.0,
  5.4,
  4.8,
  17.2,
  71.8,
  1.01,
  0.154],
 ['2023',
  'acadia',
  'Home',
  10.0,
  317.0,
  717.0,
  44.2,
  88.0,
  236.0,
  37.3,
  123.0,
  142.0,
  86.6,
  11.7,
  32.6,
  44.3,
  18.3,
  13.7,
  9.1,
  3.9,
  18.8,
  84.5,
  1.046,
  0.239],
 ['2023',
  'acadia',
  'Away',
  9.0,
  251.0,
  614.0,
  40.9,
  71.0,
  200.0,
  35.5,
  133.0,
  176.0,
  75.6,
  9.1,
  25.9,


In [103]:
fileName = 'raw_csv_files/test.csv'
#write each row of table 'rows' to row in csv file
with open(fileName,'w') as myfile:
    wrtr = csv.writer(myfile, delimiter=',')
    for row in rows:
        wrtr.writerow(row)
        myfile.flush()